In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.model_selection import GridSearchCV


In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
# print(train.head())

In [ ]:
# print(train.head())
# print(test.head())
# print(train.shape)
# print(test.shape)

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
plt.rcParams['figure.figsize']= (20,10)
sns.distplot(train['deg_C'], color='red')
plt.xlabel('Degree in celcius', size=15)

In [ ]:
plt.rcParams['figure.figsize']= (20,10)
sns.distplot(train['relative_humidity'], color='red')
plt.xlabel('Relative Humidity', size=15)

In [ ]:
plt.rcParams['figure.figsize']= (20,10)
sns.distplot(train['absolute_humidity'], color='red')
plt.xlabel('Absolute Humidity', size=15)

In [ ]:
plt.rcParams['figure.figsize']= (20,10)
sns.distplot(train['sensor_1'], color='red')
plt.xlabel('Data of sensor 1', size=15)

In [ ]:
plt.rcParams['figure.figsize']= (20,10)
sns.distplot(train['sensor_2'], color='red')
plt.xlabel('Data of sensor 2', size=15)

In [ ]:
plt.rcParams['figure.figsize']= (20,10)
sns.distplot(train['sensor_3'], color='red')
plt.xlabel('Data of sensor 3', size=15)

In [ ]:
plt.rcParams['figure.figsize']= (20,10)
sns.distplot(train['sensor_4'], color='red')
plt.xlabel('Data of sensor 5', size=15)

In [ ]:
new_train = train.drop(['target_benzene', 'date_time', 'target_carbon_monoxide', 'target_nitrogen_oxides'], axis=1) 
new_test = test.drop('date_time', axis=1)
features= ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5']
targets = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']

In [ ]:

sns.set_theme(style="whitegrid")
fig, axes = plt.subplots(4,2) # create figure and axes
fig.subplots_adjust(left=0.08, right=0.98, bottom=0.05, top=0.9,
                    hspace=0.9, wspace=0.3)

df = pd.DataFrame(data={'Degree celsuis': train['deg_C'],
                    'Relative Humidity': train['relative_humidity'],
                    'Absolute Humidity': train['absolute_humidity'],
                    'Sensor 1 Data': train['sensor_1'],
                    'Sensor 2 Data': train['sensor_2'],
                    'Sensor 3 Data': train['sensor_3'],
                    'Sensor 4 Data': train['sensor_4'],
                    'Sensor 5 Data': train['sensor_5'],  
                       })

for i,el in enumerate(list(df.columns.values)):
    a = df.boxplot(el, ax=axes.flatten()[i])

#fig.delaxes(axes[1,1]) # remove empty subplot
plt.tight_layout() 

plt.show()

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], errors='coerce')
train.loc[:, 'weekofyear'] = train['date_time'].dt.weekofyear
train.loc[:, 'month'] = train['date_time'].dt.month
train.loc[:, 'hour'] = train['date_time'].dt.hour
train['fer_C'] = (train['deg_C']*(9/5)) + 32 # in farenheit
train['kel_C'] = train['deg_C'] + 273.15     #in kelvin
train['dew_point'] = train['deg_C'] - ((100 - train['relative_humidity'])/5)

In [ ]:
test['date_time'] = pd.to_datetime(test['date_time'], errors='coerce')
test.loc[:, 'weekofyear'] = test['date_time'].dt.weekofyear
test.loc[:, 'month'] = test['date_time'].dt.month
test.loc[:, 'hour'] = test['date_time'].dt.hour
test['fer_C'] = (test['deg_C']*(9/5)) + 32
test['kel_C'] = test['deg_C'] + 273.15
test['dew_point'] = test['deg_C'] - ((100 - test['relative_humidity'])/5)

In [ ]:
y = train.loc[:,targets]

In [ ]:
y1 = np.array(y.loc[:, list(y.columns)[0] ] )# first column for prediction

y2 = np.array(y.loc[:, list(y.columns)[1] ]) # second column for prediction

y3 = np.array(y.loc[:, list(y.columns)[2] ]) # third column for prediction

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.model_selection import GridSearchCV

X_train, X_valid, y_train, y_valid = train_test_split(new_train, y1 , test_size=0.2, random_state=1)
lgbm = LGBMRegressor()
params = {
    'num_leaves': [7, 14, 21, 28, 31, 50],
    'learning_rate': [0.1, 0.03, 0.003],
    'max_depth': [-1, 3, 5],
    'n_estimators': [50, 100, 200, 500],
}
grid= GridSearchCV(estimator=lgbm, param_grid = params, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
columns = test.columns[1:]
X = train[columns].values
X_test = test[columns].values

#Since we are to predict 3 targets so we are setting target 1,2,3
#Reshaping otherwise it will throw an error
target_1 = train['target_carbon_monoxide'].values.reshape(-1,1)
target_2 = train['target_benzene'].values.reshape(-1,1)
target_3 = train['target_nitrogen_oxides'].values.reshape(-1,1)

sub = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")
sub.head()

In [ ]:
lgbm = LGBMRegressor(learning_rate= 0.1, max_depth= 5, n_estimators= 500, num_leaves= 7)

lgbm.fit(X, target_1)
sub['target_carbon_monoxide'] = lgbm.predict(X_test)

lgbm.fit(X, target_2)
sub['target_benzene'] = lgbm.predict(X_test)

lgbm.fit(X, target_3)
sub['target_nitrogen_oxides'] = lgbm.predict(X_test)

sub.head()

In [ ]:
# sub.to_csv('submission_lgbm.csv', index=False)